In [ ]:
import os
import pandas as pd 
import matplotlib.pyplot as plt 
import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np


from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split


In [ ]:
!ls

In [ ]:
import os, os.path
print(len(list(os.listdir('../input/clothing-dataset-full/images_original'))))

In [ ]:
import pandas as pd
labels = pd.read_csv('../input/clothing-dataset-full/images.csv')
labels.head()

In [ ]:
labels['label'].value_counts()

In [ ]:
labels['image'] = labels['image'] + '.jpg'
label_df = labels[['image', 'label']]
classes=list(label_df['label'].unique())
repl={}
for i in range(len(classes)):
    repl[classes[i]]=i
for i in range(len(label_df)):
    label_df['label'][i]=repl[label_df['label'][i]]

#deli=[]
#for i in range(len(label_df)):
  #if(not(os.path.isfile('input/clothing-dataset-full/images_original/'+label_df['image'][i]))):
    
    #deli.append(i)
#label_df = label_df.drop(label_df.index[deli])
    
#label_df.to_csv ('./train_csv.csv', index = False, header=True)

In [ ]:
from sklearn.model_selection import train_test_split
train_label_df, test_label_df = train_test_split(label_df, test_size=0.10)
train_label_df.to_csv ('./train_csv.csv', index = False, header=True)
test_label_df.to_csv ('./test_csv.csv', index = False, header=True)

In [ ]:
label_df

In [ ]:
from torch.utils.data import Dataset
import pandas as pd
import os
from PIL import Image
import torch

class DressDataset(Dataset):
    def __init__(self, root_dir, annotation_file, transform=None):
        self.root_dir = root_dir
        self.annotations = pd.read_csv(annotation_file)
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_id = self.annotations.iloc[index, 0]
        img = Image.open(self.root_dir+ img_id).convert("RGB")
        y_label = torch.tensor(float(self.annotations.iloc[index, 1]))

        if self.transform is not None:
            img = self.transform(img)

        return (img, y_label)

In [ ]:
transform = transforms.Compose(
    [transforms.Resize((356, 356)),transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = DressDataset(root_dir='../input/clothing-dataset-full/images_original/', annotation_file='./train_csv.csv', transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = DressDataset(root_dir='../input/clothing-dataset-full/images_original/', annotation_file='./test_csv.csv', transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class CustomConv(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(118336, 20)
        

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.fc1(x)
        return x


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Myconv2D(torch.autograd.Function):

    # Note that both forward and backward are @staticmethods
    @staticmethod
    # bias is an optional argument
    def forward(ctx, input, weight, bias=None):
        ctx.save_for_backward(input, weight, bias)
       
        # input dim [batch_size, channels, height, width]

        batch_size = len(input)
        channels = input[0][0]
        h, w = input.size(2), input.size(3)
        #image =  torch.tensor(batch_size, channels, h, w) # input image
        #print (image)
        #print (image.size())
        #print (image[0])

        kh, kw = 3, 3# kernel size
        dh, dw = 2, 2 # stride

        #filt = nn.parameter(channels, kh, kw) # filter (create this as nn.Parameter if you want to train it)
        #print (filt)
        patches = input.unfold(2, kh, dh).unfold(3, kw, dw)
        #print(patches.shape)
        patches = patches.contiguous().view(batch_size, channels, -1, kh, kw)
        #print(patches.shape)
        patches = patches.permute(0, 2, 1, 3, 4)
        patches = patches.view(-1, channels, kh, kw)
        #print (patches.shape)
        #print ("Filter shape",filt.shape)
        #print(patches)
        #print (patches,"\n","\n","\n",filt)
        #print ("Builtin multiplication")
        
        dummy = patches

        patches = patches * weight # same is done below with 4 nested loops with custom operation
        

        #print(patches)
        patches = patches.sum(1) # previous and this patch are same what is it doing
        #print (patches)
        output = patches # is it right ?
        if bias is not None:
            output += bias.unsqueeze(0).expand_as(output)
        return output

    # This function has only a single output, so it gets only one gradient
    @staticmethod
    def backward(ctx, grad_output):
        # This is a pattern that is very convenient - at the top of backward
        # unpack saved_tensors and initialize all gradients w.r.t. inputs to
        # None. Thanks to the fact that additional trailing Nones are
        # ignored, the return statement is simple even when the function has
        # optional inputs.
        input, weight, bias = ctx.saved_tensors
        grad_input = grad_weight = grad_bias = None

        # These needs_input_grad checks are optional and there only to
        # improve efficiency. If you want to make your code simpler, you can
        # skip them. Returning gradients for inputs that don't require it is
        # not an error.
        if ctx.needs_input_grad[0]:
            grad_input = grad_output.mm(weight)
        if ctx.needs_input_grad[1]:
            grad_weight = grad_output.t().mm(input)
        if bias is not None and ctx.needs_input_grad[2]:
            grad_bias = grad_output.sum(0).squeeze(0)

        return grad_input, grad_weight, grad_bias


class Myconv2D(nn.Module):
    def __init__(self):
        super(Myconv2D, self).__init__()
        self.fn = Myconv2D.apply
        # weight tensor = out_channels× in_channels/groups ×kH×kW
        self.weight = nn.Parameter(torch.randn(3, 1, 2, 2)) # when groups=1

    def forward(self, x):
        x = self.fn(x, self.weight)
        return x


In [ ]:
batch_size = 2
channels = 5
h, w = 12, 12
image = torch.randn(batch_size, channels, h, w) # input image

kh, kw = 3, 3 # kernel size
dh, dw = 3, 3 # stride

# Create conv
conv = nn.Conv2d(5, 7, (kh, kw), stride=(dh, dw), bias=False)
filt = conv.weight

# Manual approach
patches = image.unfold(2, kh, dh).unfold(3, kw, dw)
print(patches.shape) # batch_size, channels, h_windows, w_windows, kh, kw

patches = patches.contiguous().view(batch_size, channels, -1, kh, kw)
print(patches.shape) # batch_size, channels, windows, kh, kw

nb_windows = patches.size(2)

# Now we have to shift the windows into the batch dimension.
# Maybe there is another way without .permute, but this should work
patches = patches.permute(0, 2, 1, 3, 4)
print(patches.shape) # batch_size, nb_windows, channels, kh, kw

# Calculate the conv operation manually
res = (patches.unsqueeze(2) * filt.unsqueeze(0).unsqueeze(1)).sum([3, 4, 5])
print(res.shape) # batch_size, output_pixels, out_channels
res = res.permute(0, 2, 1) # batch_size, out_channels, output_pixels
# assuming h = w
h = w = int(res.size(2)**0.5)
res = res.view(batch_size, -1, h, w)

# Module approach
out = conv(image)

In [ ]:
class Polynomial3(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate four parameters and assign them as
        member parameters.
        """
        super().__init__()
        self.conv = nn.Conv2d(5, 7, (3, 3), stride=(3, 3), bias=False)
        self.weight = self.conv.weight

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        batch_size = 2
        channels = 5
        h, w = 12, 12
        image = x # input image

        kh, kw = 3, 3 # kernel size
        dh, dw = 3, 3 # stride

        # Create conv
        
        filt = self.conv.weight

        # Manual approach
        patches = image.unfold(2, kh, dh).unfold(3, kw, dw)
        print(patches.shape) # batch_size, channels, h_windows, w_windows, kh, kw

        patches = patches.contiguous().view(batch_size, channels, -1, kh, kw)
        print(patches.shape) # batch_size, channels, windows, kh, kw

        nb_windows = patches.size(2)

        # Now we have to shift the windows into the batch dimension.
        # Maybe there is another way without .permute, but this should work
        patches = patches.permute(0, 2, 1, 3, 4)
        print(patches.shape) # batch_size, nb_windows, channels, kh, kw

        # Calculate the conv operation manually
        res = (patches.unsqueeze(2) * filt.unsqueeze(0).unsqueeze(1)).sum([3, 4, 5])
        print(res.shape) # batch_size, output_pixels, out_channels
        res = res.permute(0, 2, 1) # batch_size, out_channels, output_pixels
        # assuming h = w
        h = w = int(res.size(2)**0.5)
        res = res.view(batch_size, -1, h, w)

        # Module approach
        out = self.conv(image)
        return out

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 =Polynomial3()
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(118336, 20)
        

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.fc1(x)
        return x


net = Net()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
#outputs = 0
#labels = 0
losses=[]
for epoch in range(20):  

    running_loss = 0.0
    i=0
    for inputs, labels in trainloader:
        
        optimizer.zero_grad()
        outputs = net(inputs.float())
        #break
        loss = criterion(outputs,labels.long())
        loss.backward()
        optimizer.step()        
        running_loss += loss.item()
        i+=1
        if i % 200 == 0:    
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0
    with torch.no_grad():
        val_loss=0
        for val_inputs,val_labels in testloader:
            val_outputs = net(val_inputs.float())
            #break
            loss = criterion(val_outputs,val_labels.long())     
            val_loss += loss.item()
        losses.append(val_loss)
        print("Validation loss:",val_loss)
        l=len(losses)
        if(l>1):
            if(losses[l-1]>losses[l-2]):
                break

print('Finished Training')

In [ ]:
plt.plot(losses)

In [ ]:
losses

In [ ]:
torch.save(net, "./demo1.pth")

In [ ]:
inputs=0
for inputs, labels in trainloader:
    break

In [ ]:
inputs.shape

In [ ]:
patches = inputs.unfold(2, 4, 4).unfold(3, 4, 4)
print(patches.shape)

In [ ]:
patches = patches.contiguous().view(4, 3, -1, 4, 4)
print(patches.shape)

In [ ]:
nb_windows = patches.size(2)

# Now we have to shift the windows into the batch dimension.
# Maybe there is another way without .permute, but this should work
patches = patches.permute(0, 2, 1, 3, 4)
print(patches.shape)

In [ ]:
res = (patches.unsqueeze(2) * filt.unsqueeze(0).unsqueeze(1)).sum([3, 4, 5])
print(res.shape)

In [ ]:
conv = nn.Conv2d(4, 7, (3, 3), stride=(3, 3), bias=False)
filt=conv.weight
print(filt.shape)

In [ ]:
print(filt.unsqueeze(0).shape)

In [ ]:
print(filt.unsqueeze(0).unsqueeze(1).shape)

In [ ]:
#res = (patches.unsqueeze(2) * filt.unsqueeze(0).unsqueeze(1)).sum([3, 4, 5])
print(patches.unsqueeze(2).shape)
print(filt.unsqueeze(0).unsqueeze(1).shape)

In [ ]:
patches.unsqueeze(2) * filt.unsqueeze(0).unsqueeze(1)

In [ ]:
patches = image.unfold(2, kh, dh).unfold(3, kw, dw)
        print(patches.shape) # batch_size, channels, h_windows, w_windows, kh, kw

        patches = patches.contiguous().view(batch_size, channels, -1, kh, kw)
        print(patches.shape) # batch_size, channels, windows, kh, kw

        nb_windows = patches.size(2)

        # Now we have to shift the windows into the batch dimension.
        # Maybe there is another way without .permute, but this should work
        patches = patches.permute(0, 2, 1, 3, 4)
        print(patches.shape) # batch_size, nb_windows, channels, kh, kw

        # Calculate the conv operation manually
        res = (patches.unsqueeze(2) * filt.unsqueeze(0).unsqueeze(1)).sum([3, 4, 5])
        print(res.shape) # batch_size, output_pixels, out_channels
        res = res.permute(0, 2, 1) # batch_size, out_channels, output_pixels
        # assuming h = w
        h = w = int(res.size(2)**0.5)
        res = res.view(batch_size, -1, h, w)

In [ ]:
batch_size = 1
channels = 5
h, w = 12, 12
image = torch.randn(batch_size, channels, h, w) # input image

kh, kw = 3, 3 # kernel size
dh, dw = 3, 3 # stride
print("image",image.shape)
filt = torch.randn(channels, kh, kw) # filter (create this as nn.Parameter if you want to train it)
print("filt = torch.randn(channels, kh, kw)",filt.shape) 

patches = image.unfold(2, kh, dh).unfold(3, kw, dw)
print("image.unfold(2, kh, dh)",image.unfold(2, kh, dh).shape)
print("image.unfold(2, kh, dh).unfold(3, kw, dw)",image.unfold(2, kh, dh).unfold(3, kw, dw).shape)
#> torch.Size([1, 5, 4, 4, 3, 3]) # batch_size, channels, h_windows, w_windows, kh, kw

patches = patches.contiguous().view(batch_size, channels, -1, kh, kw)
print("patches.contiguous().view(batch_size, channels, -1, kh, kw)",patches.shape) 
#> torch.Size([1, 5, 16, 3, 3]) # batch_size, channels, windows, kh, kw

# Now we have to shift the windows into the batch dimension.
# Maybe there is another way without .permute, but this should work
patches = patches.permute(0, 2, 1, 3, 4)
print("patches = patches.permute(0, 2, 1, 3, 4)",patches.shape) 
patches = patches.view(-1, channels, kh, kw)
print("patches = patches.view(-1, channels, kh, kw)",patches.shape)
#> torch.Size([16, 5, 3, 3]) # windows * batch_size, channels, kh, kw

# Now we can use our filter and sum over the channels
patches = patches * filt
print("patches = patches * filt",patches.shape,"*",filt.shape)

patches = patches.sum(1)
print(patches.shape)
#> torch.Size([16, 3, 3])